<a href="https://colab.research.google.com/github/pulkit-20-g/PSB-hack-SBI/blob/main/meta_model_sbi_hack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.linear_model import LogisticRegression


In [ ]:
df=pd.read_csv("/content/HACKATHON_TRAINING_DATA.CSV")


In [ ]:
df.head()

 INDICATES

This is a binary indicator (usually 0 or 1) that tells whether a Standing Instruction (SI) has been set on the account.

🔍 What is a Standing Instruction?
A Standing Instruction is a pre-authorized instruction from a customer to the bank to automatically debit a fixed amount at regular intervals from their account—commonly used for:

Loan EMI payments
Credit card payments
Recurring bill payments (like rent, utilities)


In [ ]:
df.shape

In [ ]:
df.describe()

easy prediction model you have been given target 0 and 1 iss pe eak neural network ko train karna hai and another walein pe test karna haii use multiple models to evaluate the accuracy which parameteres are most suitable pehle toh ye dekho

In [ ]:
arr=df.columns
for i in arr:
    print(i)

In [ ]:
for i in arr:
     print(i,df[i].dtypes)


OBJECT: AVERAGE_ACCT_AGE1
CREDIT_HISTORY_LENGTH1,INCOME_BAND1 ,AGREG_GROUP,PRODUCT_TYPE TIME_PERIOD

In [ ]:
object_cols = [
    "AVERAGE_ACCT_AGE1",
    "CREDIT_HISTORY_LENGTH1",
    "INCOME_BAND1",
    "AGREG_GROUP",
    "PRODUCT_TYPE",
    "TIME_PERIOD"
]

# Inspect unique values for each
for col in object_cols:
    print(f"\n--- {col} ---")
    print("Unique values:", df[col].unique())
    print("Value counts:\n", df[col].value_counts(dropna=False))


### changing the object type to int or float

### 1.AVERAGE_ACCT_AGE1 and CREDIT_HISTORY_LENGTH1

In [ ]:
import re

def convert_to_months(x):
    if pd.isna(x):
        return np.nan
    match = re.match(r'(\d+)yrs\s+(\d+)mon', x)
    return int(match.group(1)) * 12 + int(match.group(2)) if match else np.nan

df['AVERAGE_ACCT_AGE1'] = df['AVERAGE_ACCT_AGE1'].apply(convert_to_months)
df['CREDIT_HISTORY_LENGTH1'] = df['CREDIT_HISTORY_LENGTH1'].apply(convert_to_months)

# Optional: Impute missing with median
df['AVERAGE_ACCT_AGE1'].fillna(df['AVERAGE_ACCT_AGE1'].median(), inplace=True)
df['CREDIT_HISTORY_LENGTH1'].fillna(df['CREDIT_HISTORY_LENGTH1'].median(), inplace=True)


### 2. INCOME_BAND1

In [ ]:
income_band_map = {
    'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8,
    'EX01': 9, 'EX02': 10, 'EX04': 11, 'EX05': 12
}
df['INCOME_BAND1'] = df['INCOME_BAND1'].map(income_band_map)

# Optional: Fill NaN with median or 0
df['INCOME_BAND1'].fillna(df['INCOME_BAND1'].median(), inplace=True)


### 3. AGREG_GROUP, PRODUCT_TYPE, TIME_PERIOD

In [ ]:
# One-hot encode
df = pd.get_dummies(df, columns=['AGREG_GROUP', 'PRODUCT_TYPE', 'TIME_PERIOD'], drop_first=True)


Code to Identify Non-Numeric Columns

In [ ]:
# Get columns not of int or float dtype
non_numeric_cols = df.select_dtypes(exclude=['int64', 'float64']).columns.tolist()

print("Columns that are NOT int or float:")
for col in non_numeric_cols:
    print(f"{col} --> {df[col].dtype}")


Convert Boolean Columns to Integer

In [ ]:
# Identify boolean columns
bool_cols = df.select_dtypes(include='bool').columns

# Convert to integer type
df[bool_cols] = df[bool_cols].astype(int)

print(f"✅ Converted {len(bool_cols)} boolean columns to integers.")

check remaining columns

In [ ]:
ck=['SI_FLG','LOCKER_HLDR_IND','UID_FLG','KYC_FLG','INB_FLG','EKYC_FLG']
for col in ck:
    print(f"\n--- {col} ---")
    print("Unique values:", df[col].unique())
    print("Value counts:\n", df[col].value_counts(dropna=False))


convert to int for training

In [ ]:
# List of binary flag columns
binary_flags = ['SI_FLG', 'LOCKER_HLDR_IND', 'UID_FLG', 'KYC_FLG', 'INB_FLG', 'EKYC_FLG']

# Step 1: Drop rows where KYC_FLG is '1' or '2'
df = df[~df['KYC_FLG'].isin(['1', '2'])]

# Step 2: Convert Y/N to 1/0, leave NaNs as-is
for col in binary_flags:
    df[col] = df[col].map({'Y': 1, 'N': 0})

# Confirm changes
print("✅ Binary flags cleaned and KYC_FLG filtered. Remaining values:")
for col in binary_flags:
    print(f"{col} --> Unique values: {df[col].unique()}")


In [ ]:
# check
# Get columns not of int or float dtype
non_numeric_cols = df.select_dtypes(exclude=['int64', 'float64']).columns.tolist()

print("Columns that are NOT int or float:")
for col in non_numeric_cols:
    print(f"{col} --> {df[col].dtype}")


NOW ALL THE COLUMNS ARE INTEGER TYPE / FLOAT TYPE

In [ ]:
import pandas as pd
missing_counts = df.isnull().sum()
# Create a DataFrame for better formatting
missing_df = missing_counts.sort_values(ascending=False).reset_index()
missing_df.columns = ['Column', 'MissingValues']

# Display all rows
pd.set_option('display.max_rows', None)

print("🔍 Columns with missing values:")
print(missing_df)


## HANDLING THE NNULL VALUES

Drop high-null columns (too sparse to be useful)



In [ ]:
# Drop columns with >150,000 missing values (roughly 45%+ missing)
high_null_cols = df.columns[df.isnull().sum() > 150000]
df.drop(columns=high_null_cols, inplace=True)
print(f"Dropped high-null columns: {list(high_null_cols)}")

Fill monthly transaction behavior columns with 0 (assumes "no activity")

In [ ]:
monthly_cols = [col for col in df.columns if any(key in col for key in [
    'MNTHSCR', 'MNTHSDR', 'AVGMTD', 'AVGQTD', 'AVGYTD', 'OUTSTANGBAL'])]

df[monthly_cols] = df[monthly_cols].fillna(0)


3. Impute credit bureau and CRIFF columns with media (skewed distributions)



In [ ]:
bureau_cols = [col for col in df.columns if any(key in col for key in [
    'CRIFF_', 'PRI_', 'PRIMARY_INSTAL_AMT', 'TOTAL_CRIFF1', 'DEC_CRIFFCHNG1', 'NO_OF_INQUIRIES1'])]

df[bureau_cols] = df[bureau_cols].fillna(df[bureau_cols].median())


4. Impute general numeric columns (e.g., AGE, tenure fields) with median



In [ ]:
general_cols = ['AGE', 'VINTAGE', 'KYC_SCR', 'ACCT_RESIDUAL_TENURE',
                'LATEST_RESIDUAL_TENURE', 'OLDEST_RESIDUAL_TENURE',
                'LATEST_CR_DAYS', 'LATEST_RG3_TENURE', 'NO_YRS_RG3']

df[general_cols] = df[general_cols].fillna(df[general_cols].median())


5. Drop rows with remaining minimal nulls in critical flag columns

In [ ]:
final_check_cols = ['UID_FLG', 'KYC_FLG', 'INB_FLG', 'EKYC_FLG', 'LOCKER_HLDR_IND', 'POP_CODE']

# Drop rows if any of these critical flags are still null
df.dropna(subset=final_check_cols, inplace=True)


In [ ]:
missing_cols = df.columns[df.isnull().any()]
print("Columns with missing values:")
print(df[missing_cols].isnull().sum())


In [ ]:
fill_zero_cols = [
    'ONEMNTHCR',
    'LAST_1_YR_RG2',
    'LAST_1_YR_RG1',
    'NEW_ACCTS_IN_LAST_SIX_MONTHS1',
    'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS1'
]

# Fill all with 0
df[fill_zero_cols] = df[fill_zero_cols].fillna(0)


In [ ]:
print("Remaining null values:", df.isnull().sum().sum())


Outlier Handling Code (Safe for Large Datasets)

In [ ]:
# Step 1: Select only continuous numeric columns (exclude flags, indicators, dummies)
excluded_keywords = ['FLG', 'IND', 'CODE', 'TYPE', 'TENURE', 'NO_', 'KYC', 'CRIFF', 'RG', 'POP', 'TIME', 'TARGET']
numeric_cols = [col for col in df.columns
                if df[col].dtype in ['float64', 'int64']
                and not any(key in col.upper() for key in excluded_keywords)]

print(f"✅ Number of numeric columns selected for outlier clipping: {len(numeric_cols)}")

# Step 2: Clip outliers at the 1st and 99th percentile for each column
for col in numeric_cols:
    lower = df[col].quantile(0.01)
    upper = df[col].quantile(0.99)
    df[col] = df[col].clip(lower, upper)

print("✅ Outlier clipping complete using 1st–99th percentile.")


### NOW LETS START MODEL TRAINING

In [ ]:
X = df.drop(columns=['TARGET'])  # Features
y = df['TARGET']                 # Target (0 = non-defaulter, 1 = defaulter)

In [ ]:
df_pred=pd.read_csv('/content/HACKATHON_PREDICTION_DATA.CSV')

In [ ]:
df_pred.head()

ANALYSE df_pred

In [ ]:
print("Shape:", df_pred.shape)
print("Columns:", df_pred.columns.tolist()[:10], "...")

# Check if 'TARGET' is present (it shouldn't be)
if 'TARGET' in df_pred.columns:
    print("TARGET column found in prediction data — please verify!")
else:
    print("No TARGET column in prediction data — looks good.")

# Compare column names with training set (assuming df already cleaned)
common_cols = [col for col in df.columns if col in df_pred.columns]
missing_in_pred = [col for col in df.columns if col not in df_pred.columns]

print(f"\nCommon columns with training set: {len(common_cols)}")
print(f"Missing columns in prediction set: {missing_in_pred}")

# Check for nulls
null_summary = df_pred.isnull().sum()
null_summary = null_summary[null_summary > 0].sort_values(ascending=False)

print("\n🔍 Columns with missing values in df_pred:")
print(null_summary.to_string())

# Summary stats of numeric features
print("\n📊 Summary statistics of numeric columns:")
print(df_pred.describe().T)


In [ ]:
print("Shape:", df_pred.shape)
print("Columns:", df_pred.columns.tolist()[:10], "...")

common_cols = [col for col in df.columns if col in df_pred.columns]
missing_in_pred = [col for col in df.columns if col not in df_pred.columns]


In [ ]:
df_pred.describe().T


### ANALYSED THE PREDICTION DATA SET NEED TO DO CLEANING AND ALL OF THIS ALSO BUT FIRST LETS BUILD UP ON THE MODEL AND THEN WILL DO THIS

LETS CHECK IF THE DATA SET IS IMBALANCED OR NOT??

In [ ]:
# Check target distribution
import seaborn as sns
import matplotlib.pyplot as plt

target_counts = df['TARGET'].value_counts(normalize=True) * 100
print("🔢 Class Distribution (%):")
print(target_counts)

# Optional: Bar plot
plt.figure(figsize=(6, 4))
sns.countplot(data=df, x='TARGET')
plt.title('Target Variable Distribution')
plt.xlabel('Defaulter (1 = Yes, 0 = No)')
plt.ylabel('Count')
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import layers, models
import tensorflow as tf

In [ ]:
X = df.drop(columns=['TARGET'])
y = df['TARGET']

# STEP 2: Train-Test Split (Stratified for imbalance)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)


In [ ]:
# STEP 4: Compute Class Weights
class_weights_array = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights_array))
print("✅ Class Weights:", class_weight_dict)

# STEP 5: Build the Neural Network Model
def build_nn_model():
    model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
# STEP 4: Compute Class Weights
class_weights_array = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights_array))
print("✅ Class Weights:", class_weight_dict)

# STEP 5: Build the Neural Network Model
def build_nn_model():
    model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# STEP 6: Compile the Model
    model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)
    return model

# STEP 7: Train the Model with Validation
nn = build_nn_model()
history = nn.fit(
    X_train_scaled, y_train,
    epochs=50,
    batch_size=512,
    validation_data=(X_val_scaled, y_val),
    class_weight=class_weight_dict,
    verbose=2
)

# STEP 8: Save the Model
nn.save("defaulter_model.h5")
print("📦 Model saved as defaulter_model.h5")


In [ ]:
from sklearn.model_selection import KFold
import numpy as np
from tensorflow.keras import layers, models
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

# Assuming X and y are already defined and preprocessed from the previous steps
# X = df.drop(columns=['TARGET'])  # Features
# y = df['TARGET']                 # Target

# --- Cross-Validation Setup ---
n_splits = 5  # Define the number of folds for cross-validation
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# --- Store Results ---
fold_results = []
all_confusion_matrices = []
all_classification_reports = []

# --- K-Fold Cross-Validation Loop ---
fold_no = 1
for train_index, val_index in kf.split(X):
    print(f"--- Processing Fold {fold_no}/{n_splits} ---")

    # Split data into training and validation sets for the current fold
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # Scale the data for the current fold
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # Compute Class Weights for the current fold's training data (optional but recommended for imbalance)
    class_weights_array = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
    class_weight_dict = dict(enumerate(class_weights_array))
    print("✅ Class Weights for this fold:", class_weight_dict)

    # Build the Neural Network Model
    # Model architecture is defined inside the loop to ensure a fresh start for each fold
    model = models.Sequential([
        layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

    # Compile the Model
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )

    # Train the Model
    history = model.fit(
        X_train_scaled, y_train,
        epochs=50, # You might adjust epochs based on monitoring validation performance
        batch_size=512,
        validation_data=(X_val_scaled, y_val),
        class_weight=class_weight_dict,
        verbose=0 # Set to 1 or 2 for more detailed output during training
    )

    # Evaluate the Model on the validation set for the current fold
    loss, accuracy, auc = model.evaluate(X_val_scaled, y_val, verbose=0)
    print(f"Fold {fold_no} Evaluation: Loss={loss:.4f}, Accuracy={accuracy:.4f}, AUC={auc:.4f}")

    # Store results for this fold
    fold_results.append({'fold': fold_no, 'loss': loss, 'accuracy': accuracy, 'auc': auc})

    # Generate and store confusion matrix and classification report for this fold
    y_val_pred_prob = model.predict(X_val_scaled, verbose=0)
    y_val_pred = (y_val_pred_prob > 0.5).astype(int) # Using 0.5 as threshold

    cm = confusion_matrix(y_val, y_val_pred)
    all_confusion_matrices.append({'fold': fold_no, 'confusion_matrix': cm})

    report = classification_report(y_val, y_val_pred, target_names=['Non-Defaulter', 'Defaulter'], output_dict=True)
    all_classification_reports.append({'fold': fold_no, 'report': report})

    fold_no += 1

# --- Summarize Results ---
print("\n--- Cross-Validation Summary ---")
results_df = pd.DataFrame(fold_results)
print(results_df)

print("\nMean Performance Across Folds:")
print(results_df[['accuracy', 'auc', 'loss']].mean())

# You can further analyze the stored confusion matrices and classification reports
# For example, to see the report for the first fold:
# print("\nClassification Report for Fold 1:")
# print(all_classification_reports[0]['report'])

In [ ]:
!pip install scikeras[tensorflow]

In [ ]:
# 9. Train LightGBM model
import lightgbm as lgb
lgbm = lgb.LGBMClassifier(n_estimators=200, class_weight='balanced', random_state=42)
lgbm.fit(X_train_scaled, y_train)

# 10. Wrap NN for stacking
from scikeras.wrappers import KerasClassifier
nn_wrapper = KerasClassifier(build_fn=build_nn_model,
                             epochs=50,
                             batch_size=512,
                             class_weight=class_weight_dict,
                             verbose=0)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    precision_recall_curve,
    precision_score,
    recall_score,
    roc_auc_score
)

# 1) Prepare storage for out-of-fold predictions
n_train = X_train_scaled.shape[0]
oof_probs = np.zeros((n_train, 2))  # col0: NN, col1: LGBM

kf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_scaled, y_train)):
    # a) Train fresh copies on the train-fold
    nn_fold = build_nn_model()
    nn_fold.fit(
        X_train_scaled[train_idx],
        y_train.iloc[train_idx],         # ← use original y_train here
        epochs=50,
        batch_size=512,
        class_weight=class_weight_dict,
        verbose=0
    )

    lgbm_fold = lgb.LGBMClassifier(
        n_estimators=200,
        class_weight='balanced',
        random_state=42
    )
    lgbm_fold.fit(
        X_train_scaled[train_idx],
        y_train.iloc[train_idx]          # ← and here
    )

    # b) Predict on the fold’s validation slice
    oof_probs[val_idx, 0] = nn_fold.predict(X_train_scaled[val_idx]).ravel()
    oof_probs[val_idx, 1] = lgbm_fold.predict_proba(X_train_scaled[val_idx])[:, 1]

# 2) Train meta-learner on those OOF predictions
meta = LogisticRegression(class_weight='balanced', max_iter=1000)
meta.fit(oof_probs, y_train.values)

# 3) Build meta-features for your held-out validation set
val_probs_nn   = nn.predict(X_val_scaled).ravel()
val_probs_lgbm = lgbm.predict_proba(X_val_scaled)[:, 1]
meta_features  = np.vstack([val_probs_nn, val_probs_lgbm]).T

# 4) Meta-model’s final probabilities
final_probs = meta.predict_proba(meta_features)[:, 1]

# 5) Threshold tuning
precision, recall, thresholds = precision_recall_curve(y_val, final_probs)
target_recall = 0.85
ix = np.argmin(np.abs(recall - target_recall))
opt_thresh = thresholds[ix]

# 6) Evaluate at that threshold
y_pred = (final_probs >= opt_thresh).astype(int)
print(f"Blended Precision: {precision_score(y_val, y_pred):.3f}")
print(f"Blended Recall   : {recall_score   (y_val, y_pred):.3f}")
print(f"Blended ROC AUC  : {roc_auc_score(y_val, final_probs):.3f}")



In [ ]:
# 7) Save the blended pipeline
import joblib
joblib.dump({
    'nn_model'  : nn,
    'lgbm_model': lgbm,
    'meta_model': meta,
    'scaler'    : scaler,
    'threshold' : opt_thresh
}, 'defaulter_blend.pkl')
print("✅ Blended pipeline saved to defaulter_blend.pkl")


In [ ]:
from sklearn.metrics import classification_report

# assuming you’ve already got:
#   final_probs = meta.predict_proba(meta_features)[:,1]
#   opt_thresh  = your chosen threshold (e.g. for 85% recall)
#   y_pred      = (final_probs >= opt_thresh).astype(int)

print(classification_report(
    y_val,
    y_pred,
    target_names=['Non-Defaulter','Defaulter']
))


USE PRECISION RECALL TRADEOFF (OPTIONAL)


**CONFUSION MATRIX**

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

# 1. Assuming you already have:
#    final_probs = meta.predict_proba(meta_features)[:, 1]
#    opt_thresh  = your chosen threshold (e.g. ~0.37)
#    y_pred      = (final_probs >= opt_thresh).astype(int)

# 2. Compute the confusion matrix
cm = confusion_matrix(y_val, y_pred)
tn, fp, fn, tp = cm.ravel()

print("Confusion matrix:")
print(cm)
print(f"True Negatives : {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")
print(f"True Positives : {tp}")

# 3. (Optional) Plot it with Matplotlib
fig, ax = plt.subplots()
im = ax.imshow(cm, cmap='Blues', interpolation='nearest')
ax.set_xticks([0,1])
ax.set_yticks([0,1])
ax.set_xticklabels(['Non-Defaulter', 'Defaulter'])
ax.set_yticklabels(['Non-Defaulter', 'Defaulter'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')

# annotate each cell
for i in range(2):
    for j in range(2):
        ax.text(j, i, cm[i, j], ha='center', va='center')

plt.colorbar(im, ax=ax)
plt.show()
